In [1]:
import os
import pandas as pd 
import numpy as np 

# visualisation
import matplotlib.pylab as plt
import matplotlib.font_manager as fm

# our scripts 
import src.processing as sp 
import settings as st
import utils.helpers as hp

### Correct for the locations of the images 

In [ ]:
# dr5_desc = sp.read_parquet('gz_decals_volunteers_5', save = True, filename='decals_5_votes')

### Generate the descriptions for spiral and elliptical 

In [2]:
# dr5_desc = hp.read_parquet(st.data_dir, 'descriptions/decals_5_votes')

In [4]:
# dr5_spiral = sp.filtering(dr5_desc, st.condition_spiral, save=True, filename = 'decals_5_spiral')

In [5]:
# dr5_elliptical = sp.filtering(dr5_desc, st.condition_elliptical, save=True, filename = 'decals_5_elliptical')

### Subset of the Spirals and Ellipticals 

In [12]:
# df_spiral= sp.subset_df(dr5_spiral, 2000, random=True, save=True, filename='subset_spiral')

In [14]:
# df_elliptical = sp.subset_df(dr5_elliptical, 2000, random=True, save=True, filename='subset_elliptical')

### Move data
Given a pandas dataframe, copy images from Mike's folder to our folder.

In [12]:
# df_spiral = hp.read_parquet(st.data_dir, 'descriptions/subset_spiral')
# df_elliptical = hp.read_parquet(st.data_dir, 'descriptions/subset_elliptical')

In [13]:
# sp.copy_images(df_spiral, 'spiral')
# sp.copy_images(df_elliptical, 'elliptical')

### Split the data into Training/Validation

In [28]:
# d = sp.split_data(['spiral', 'elliptical'], save=False)

### Generate the Training/Validation sets (images)

In [2]:
# sp.images_train_validate(['spiral', 'elliptical'])